# Getting the data

In [17]:
from pyspark import SparkContext, SparkConf
from math import sqrt,log
from random import randint, random


sc = SparkContext.getOrCreate()
sc.stop()

conf = SparkConf().setAppName("kmeans").setMaster("local[*]")
sc = SparkContext(conf=conf)

* Getting data from S-sets : https://cs.joensuu.fi/sipu/datasets/
* Using the s1 dataset

In [18]:
s1 = sc.textFile("../data/s1.txt")


def extract_split(x):
    splits = x.split('    ')
    return (int(splits[1]), int(splits[2]))

s1_map = s1.flatMap(lambda x: x.split('\n')).map(extract_split)



In [19]:
s1_map.takeSample(False, 20)

[(394599, 409068),
 (420489, 448078),
 (257472, 849587),
 (603855, 575180),
 (293746, 544468),
 (677551, 860007),
 (329859, 564384),
 (289176, 571435),
 (169628, 365003),
 (448651, 847994),
 (410682, 349328),
 (722463, 882675),
 (150009, 371014),
 (920249, 551171),
 (386068, 422517),
 (680969, 838864),
 (598671, 854219),
 (257134, 845851),
 (569197, 559307),
 (246857, 852483)]

# Kmeans ++ intitialization

In [20]:
cluster1_center = s1_map.takeSample(False, 1)[0]


data_length = s1_map.count()
print(data_length)
BUCKET_SIZE = 1000
BUCKET_NUMBER = int(data_length / BUCKET_SIZE)

print(BUCKET_NUMBER)

5000
5


In [21]:
s1_map = s1_map.map(lambda x:(randint(1,BUCKET_NUMBER),) + x)

In [22]:
s1_map = s1_map.map(lambda x : x + cluster1_center)
s1_map.takeSample(False,20)

[(1, 439514, 758221, 186536, 330019),
 (2, 894124, 202052, 186536, 330019),
 (4, 554938, 347875, 186536, 330019),
 (5, 619628, 399433, 186536, 330019),
 (2, 115781, 594942, 186536, 330019),
 (5, 296775, 572432, 186536, 330019),
 (5, 906396, 535803, 186536, 330019),
 (1, 669430, 905798, 186536, 330019),
 (2, 300726, 862728, 186536, 330019),
 (3, 142886, 556511, 186536, 330019),
 (1, 553038, 606418, 186536, 330019),
 (4, 572244, 349463, 186536, 330019),
 (2, 849858, 524682, 186536, 330019),
 (1, 239387, 847822, 186536, 330019),
 (4, 412111, 418466, 186536, 330019),
 (2, 868025, 569296, 186536, 330019),
 (1, 317136, 563479, 186536, 330019),
 (2, 879221, 655497, 186536, 330019),
 (4, 284077, 905181, 186536, 330019),
 (4, 105568, 584102, 186536, 330019)]

In [23]:
def compute_distance(xi_indexes, yi_indexes):
    def bar(row):
        sum = 0
        for i in range(len(xi_indexes)):
            sum += (row[yi_indexes[i]] - row[xi_indexes[i]]) ** 2
        distance = sqrt(sum)
        return row + (distance,)
    return bar


In [24]:
def reduceByKeyMax(dist_indexes):
    def reduce_custom(x1,x2):
        dist_x1 = []
        dist_x2 = []
        for idx in dist_indexes:
            dist_x1.append(x1[idx])
            dist_x2.append(x2[idx])
        
        mindist_x1 =  min(dist_x1)
        mindist_x2 = min(dist_x2)
        
        '''
        if(mindist_x1 > mindist_x2):
            return x1
        else:
            return x2
        '''
        #Another version with more randomness => should converge less
        if(mindist_x1 != 0):
            drawx1 = log(mindist_x1) * random() 
        else:
            drawx1 = 0
        if(mindist_x2 != 0):
            drawx2 = log(mindist_x2) * random()
        else:
            drawx2 = 0
            
            
        if(drawx1 > drawx2):
            return x1
        else:
            return x2
        
        
    return reduce_custom



In [25]:
def compute_average(list_reduce, coord_indexes):
    result = ()
    for idx in coord_indexes:
        coord_list = [reduce_tuple[1][idx] for reduce_tuple in list_reduce]
        average = sum(coord_list)/len(coord_list)
        result += (average,)
    return result
        

In [26]:
def initCluster(data,num_clusters,num_features):
    xi_indexes = [idx for idx in range(1,num_features+1)]
    yi_indexes = [idx for idx in range(num_features+1, num_features*2 + 1)]
    coord_indexes = [idx for idx in range(num_features)]
    dist_indexes = [num_features*2]
    current_clust = 2        
    for _ in range(num_clusters-1):
        
        
        data = data.map(compute_distance(xi_indexes, yi_indexes))
        
        reduce_tuple = data.map(lambda x:(x[0], x[1:]))
        
        reduce_tuple = reduce_tuple.reduceByKey(reduceByKeyMax(dist_indexes=dist_indexes))
        
        new_cluster_point = compute_average(reduce_tuple.collect(), coord_indexes=coord_indexes)
        
        print("Computing center of cluster n° {}".format(current_clust))
        print("New cluster point : {}".format(new_cluster_point))
        
        data = data.map(lambda x:(x + new_cluster_point))
        
        current_clust += 1
        dist_indexes.append(current_clust*num_features + current_clust - 2)
        yi_indexes = [old_value + num_features + 1 for old_value in yi_indexes]
        
        print("New yi indexes : {}".format(yi_indexes))
        print("New dist_indexes : {}".format(dist_indexes))
        print("\n")

    return data
        


In [27]:
s1_map = initCluster(s1_map, num_features=2,num_clusters=20)
s1_map.takeSample(withReplacement=False,num=100)


Computing center of cluster n° 2
New cluster point : (637987.0, 600534.4)
New yi indexes : [6, 7]
New dist_indexes : [4, 7]


Computing center of cluster n° 3
New cluster point : (624154.6, 683058.6)
New yi indexes : [9, 10]
New dist_indexes : [4, 7, 10]


Computing center of cluster n° 4
New cluster point : (648246.6, 694693.2)
New yi indexes : [12, 13]
New dist_indexes : [4, 7, 10, 13]


Computing center of cluster n° 5
New cluster point : (623752.0, 592913.0)
New yi indexes : [15, 16]
New dist_indexes : [4, 7, 10, 13, 16]


Computing center of cluster n° 6
New cluster point : (628433.2, 572522.0)
New yi indexes : [18, 19]
New dist_indexes : [4, 7, 10, 13, 16, 19]


Computing center of cluster n° 7
New cluster point : (651461.2, 769708.8)
New yi indexes : [21, 22]
New dist_indexes : [4, 7, 10, 13, 16, 19, 22]


Computing center of cluster n° 8
New cluster point : (640838.2, 661623.4)
New yi indexes : [24, 25]
New dist_indexes : [4, 7, 10, 13, 16, 19, 22, 25]


Computing center of clu

[(1,
  451670,
  388347,
  186536,
  330019,
  271474.11209910974,
  623871.6,
  498011.0,
  204155.78350504793,
  623871.6,
  498011.0,
  204155.78350504793,
  623871.6,
  498011.0,
  204155.78350504793,
  623871.6,
  498011.0,
  204155.78350504793,
  623871.6,
  498011.0,
  204155.78350504793,
  623871.6,
  498011.0,
  204155.78350504793,
  623871.6,
  498011.0,
  204155.78350504793,
  623871.6,
  498011.0,
  204155.78350504793,
  623871.6,
  498011.0,
  204155.78350504793,
  623871.6,
  498011.0,
  204155.78350504793,
  623871.6,
  498011.0,
  204155.78350504793,
  623871.6,
  498011.0,
  204155.78350504793,
  623871.6,
  498011.0,
  204155.78350504793,
  623871.6,
  498011.0,
  204155.78350504793,
  623871.6,
  498011.0,
  204155.78350504793,
  623871.6,
  498011.0,
  204155.78350504793,
  623871.6,
  498011.0,
  204155.78350504793,
  623871.6,
  498011.0,
  204155.78350504793,
  623871.6,
  498011.0),
 (4,
  404273,
  429948,
  186536,
  330019,
  239572.9663588945,
  623871.6,
  

# Vieux brouillon de code en dessous

In [28]:
s1_map = s1_map.map(compute_distance([1,2],[3,4]))
s1_map.takeSample(False,20)

[(3,
  215976,
  836025,
  186536,
  330019,
  506861.7026724351,
  623871.6,
  498011.0,
  529747.3781863955,
  623871.6,
  498011.0,
  529747.3781863955,
  623871.6,
  498011.0,
  529747.3781863955,
  623871.6,
  498011.0,
  529747.3781863955,
  623871.6,
  498011.0,
  529747.3781863955,
  623871.6,
  498011.0,
  529747.3781863955,
  623871.6,
  498011.0,
  529747.3781863955,
  623871.6,
  498011.0,
  529747.3781863955,
  623871.6,
  498011.0,
  529747.3781863955,
  623871.6,
  498011.0,
  529747.3781863955,
  623871.6,
  498011.0,
  529747.3781863955,
  623871.6,
  498011.0,
  529747.3781863955,
  623871.6,
  498011.0,
  529747.3781863955,
  623871.6,
  498011.0,
  529747.3781863955,
  623871.6,
  498011.0,
  529747.3781863955,
  623871.6,
  498011.0,
  529747.3781863955,
  623871.6,
  498011.0,
  529747.3781863955,
  623871.6,
  498011.0,
  529747.3781863955,
  623871.6,
  498011.0,
  506861.7026724351),
 (5,
  152584,
  564548,
  186536,
  330019,
  236973.81742504804,
  623871.6,

In [29]:

s1_reduce = s1_map.map(lambda x:(x[0], x[1:6]))
s1_reduce.takeSample(False,20)


[(1, (837032, 509264, 186536, 330019, 674739.8135881711)),
 (4, (769876, 257669, 186536, 330019, 587809.5593812676)),
 (1, (525670, 184762, 186536, 330019, 368932.87466014735)),
 (5, (832086, 716249, 186536, 330019, 752268.84516109)),
 (5, (328438, 532686, 186536, 330019, 247406.7308967159)),
 (2, (406184, 394506, 186536, 330019, 228918.8001737734)),
 (5, (202627, 179376, 186536, 330019, 151499.94630362085)),
 (4, (343717, 559909, 186536, 330019, 278487.4842088958)),
 (4, (198415, 321418, 186536, 330019, 14665.87338006162)),
 (5, (117585, 391892, 186536, 330019, 92641.82926734554)),
 (4, (236749, 813868, 186536, 330019, 486447.5307471506)),
 (4, (284031, 566654, 186536, 330019, 255932.4095342362)),
 (4, (603698, 573585, 186536, 330019, 483061.62194900145)),
 (2, (338084, 560295, 186536, 330019, 275669.7888416502)),
 (1, (607269, 536961, 186536, 330019, 468872.31593793206)),
 (5, (605310, 394011, 186536, 330019, 423635.03058647073)),
 (4, (179472, 342273, 186536, 330019, 14144.278419205

In [30]:
s1_reduce = s1_reduce.reduceByKey(reduceByKeyMax(dist_indexes=[4]))
s1_reduce.collect()

[(2, (684091, 842566, 186536, 330019, 714328.6395168543)),
 (4, (624343, 403629, 186536, 330019, 443952.0259543817)),
 (1, (605891, 428146, 186536, 330019, 430682.6257860886)),
 (3, (657450, 365008, 186536, 330019, 472212.05566673115)),
 (5, (665426, 853940, 186536, 330019, 709809.0210338271))]

In [31]:
print(cluster1_center)
print (new_cluster_point)

(186536, 330019)


NameError: name 'new_cluster_point' is not defined

In [ ]:
new_cluster_point = compute_average(s1_reduce.collect(), [0,1])
